EXTRACCIÓN DATOS DE ESTACIONES METEOROLÓGICAS AEMET DE COMUNIDAD VALENCIANA
Con este script vamos a extraer datos históricos de estaciones meteorológicas reales de la comunidad valenciana.
Vamos a utilizar la api de AEMET que se encuentra en la URL: https://opendata.aemet.es/dist/index.html?

CREDENCIALES
en AWS tendremos las credenciales de AWS
#export AWS_ACCESS_KEY_ID=aws_access_key_id
#export AWS_SECRET_ACCESS_KEY=aws_secret_access_key
#export AWS_SESSION_TOKEN=aws_session_tokeN
##nombre del Bucket S3
#export BUCKET_NAME = BUCKET_NAME
##Api_Key de la api
#export api_key = api_key

In [ ]:
#Para leer las credenciales 
import os
import boto3

# Obtenemos las credenciales de las variables de entorno
aws_access_key_id = os.environ.get('AWS_ACCESS_KEY_ID')
aws_secret_access_key = os.environ.get('AWS_SECRET_ACCESS_KEY')
aws_session_token = os.environ.get('AWS_SESSION_TOKEN')
#Lectura del bucket de S3
BUCKET_NAME = os.environ.get('BUCKET_NAME')
# Obtenemos las API keys de las variables de entorno
api_key = os.environ.get('api_key')

In [37]:
import requests #para trabajar con APIs
import pandas as pd #para el tratamiento de datos con pandas
import json #para trabajar con ficheros json
import boto3 #para interactuar con AWS (concretamente con S3)
import csv #para importar el csv de las estaciones

In [38]:
#Trabajo con fechas
from dateutil.parser import parse #para convertir texto en fechas
from datetime import timedelta #para poder operar con fechas

In [14]:
#Lectura de credenciales para s3
s3 = boto3.client(
 's3',
 aws_access_key_id=aws_access_key_id,
 aws_secret_access_key=aws_secret_access_key,
 aws_session_token=aws_session_token
)

In [40]:
#headers para la API de Aemet
headers = {
    'accept': 'application/json',
    'api_key': api_key}

Cada estación tiene una fecha de comienzo diferente, 
por eso tenemos que indicar la fecha de comienzo para cada estación
Cada ciudad tiene una estación meteorológica con un id (idema)
Para ello creamos un fichero csv con el nombre de las ciudades, su id, y sus fechas de inicio
"parametros_todas_estaciones.csv"
Para poder leer todas, tenemos que crear un bucle de lectura de esos parametros y guardarlos en una lista

In [41]:
#creación de la lista con los parametros de ciudades, idema y fechas
parametros = []

#Bucle para la lectura de todas las estaciones de la Comunidad valenciana
# Abrir el archivo CSV
with open('parametros_todas_estaciones.csv', newline='') as csvfile:
    # Crear un lector de CSV con delimitador de tabulación
    lector_csv = csv.reader(csvfile)
    # Saltar la primera fila si contiene encabezados
    next(lector_csv)
    # Leer cada fila del archivo CSV
    for fila in lector_csv:
        # Agregar los parámetros a la lista
        parametros.append(fila)

La Api solo permite coger datos de 5 años. 
Para conseguir todo el histórico tenemos que ir incrementando la fecha
Para no descuadrar fechas debido a los años bisiestos, cogemos datos cada 4 años

Tendremos por tanto un bucle donde se irán descargando los datos de la API cada 4 años
Se leen los datos en json y los vamos convirtiendo en ficheros parquet que se almacenan en s3

In [ ]:
# Iterar sobre la lista de parámetros
for ciudad, idema, fechaIniStr, fechaFinStr in parametros:
    # Convertir las cadenas de fecha a objetos datetime
    fechaInit = parse(fechaIniStr.replace('%3A', ':'))
    fechaFint = parse(fechaFinStr.replace('%3A', ':'))
    # Este es el bucle para sumar 4 años a cada fecha
    i=0
    fechaFin= '2023-12-31T23:59:59UTC'
    while fechaFin<= '2023-12-31T23:59:59UTC':
        #Ciclos de 4 años
        fechaInit += timedelta(days=365*3+366)
        fechaIni = fechaInit.strftime('%Y-%m-%dT%H:%M:%S%Z')
        fechaFint += timedelta(days=365*3+366)
        fechaFin =fechaFint.strftime('%Y-%m-%dT%H:%M:%S%Z')
        
        #lectura de la API de AEMET
        url = "https://opendata.aemet.es/opendata/api/valores/climatologicos/diarios/datos/fechaini/{}/fechafin/{}/estacion/{}".format(fechaIni, fechaFin, idema)
        res = requests.get(url, headers=headers)

        #los datos están en una nueva url, así que hay que hacer una segunda lectura
        res_json = res.json()
        url_2 =res_json["datos"] 
        res = requests.get(url_2, headers=headers) #lectura de los datos de la segunda url

        #conversión de los datos en dataframe
        res_json = res.json()
        datos_json_str = json.dumps(res_json) #convierte los datos en json
        df = pd.read_json(datos_json_str) #convierte los datos en dataframe

        #conversión de datos en csv y parquet
        #df.to_csv(f'df_{ciudad}{i+1}.csv') #convierte el datframe en csv para comprobaciones
        df.to_parquet(f'df_{ciudad}{i+1}.parquet') #convierte el dataframe en parquet

        #Almacenamos los datos en el bucket de S3
        #s3.upload_file(f'df_{i+1}.parquet', BUCKET_NAME, f'rawalicante/df_{i+1}.parquet')
        i += 1
    